Basic self attention
- calculate dot product bt embeddings for each token in sequence and query (selected token at this time)
- normalize this to basically calculate attention scores of each input token
- multiply token embedding and attention score to get attention weights


In [2]:
import torch

input_embeddings = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

use extremely low dimension embeddings for illustration

In [3]:
query = input_embeddings[1] # journey
attention_scores_2 = torch.empty(input_embeddings.shape[0])
for i, token_embedding in enumerate(input_embeddings):
    attention_scores_2[i] = torch.dot(query, token_embedding)
print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
#simple normalization
attention_weights_2 = attention_scores_2/attention_scores_2.sum()
print(attention_weights_2)

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [6]:
#normalization with softmax
attention_weights_2 = torch.softmax(attention_scores_2, dim=0)
print(attention_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [7]:
context_vector_2 = torch.zeros(query.shape[0])
for i, token_embedding in enumerate(input_embeddings):
    context_vector_2 += attention_weights_2[i] * token_embedding
print(context_vector_2)

tensor([0.4419, 0.6515, 0.5683])


### Calculate attention weights for all input tokens